# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

In [1]:
%load_ext autoreload
%autoreload 2

from task1_lib import gabriele, pure_random, strategy_v3
from task2_lib import run_GA, strategy_ga
from nim_utils import evaluate, evaluate_GA, play_match

# Task 1

In [2]:
NUM_MATCHES = 1000
NIM_SIZE = 10
K_SIZE = None

#print(f"Win-rate against {gabriele.__name__}: {evaluate(strategy_v3, gabriele, NUM_MATCHES, NIM_SIZE, k_size=K_SIZE)}")
#print(f"Win-rate against {pure_random.__name__}: {evaluate(strategy_v3, pure_random, NUM_MATCHES, NIM_SIZE, k_size=K_SIZE)}")


# Task 2

In [5]:
best_genome = run_GA()
print(f"Win-rate against gabriele: {evaluate_GA(best_genome, strategy_ga, gabriele, NUM_MATCHES, NIM_SIZE, k_size=K_SIZE)}")
print(f"Win-rate against pure_random: {evaluate_GA(best_genome, strategy_ga, pure_random, NUM_MATCHES, NIM_SIZE, k_size=K_SIZE)}")

[info] - Start generating the population


100%|██████████| 30/30 [00:00<00:00, 45.51it/s]


[info] - Evolving...


 21%|██        | 21/100 [00:05<00:21,  3.59it/s]

[info] - best.fitness = 1.0
[info] - avg.fitness = 1.0


...

[info] - best.fitness = 1.0
[info] - avg.fitness = 1.0


 81%|████████  | 81/100 [00:23<00:05,  3.46it/s]

[info] - best.fitness = 1.0
[info] - avg.fitness = 1.0


100%|██████████| 100/100 [00:28<00:00,  3.49it/s]


[info] - Best genome found is {'alpha': 0.9231869349607998, 'beta': 0.0441073349783464} with fitness: 1.0
Win-rate against gabriele: 1.0
Win-rate against pure_random: 0.921


# Task 3

# Task 4

## Oversimplified match

In [4]:
play_match(strategy_v3, pure_random, 10, k_size=None)

status: After player 0 -> <1 3 1 7 9 11 13 15 17 19>
status: After player 1 -> <1 3 1 7 9 11 13 5 17 19>
status: After player 0 -> <1 3 1 7 1 11 13 5 17 19>
status: After player 1 -> <1 3 1 7 1 1 13 5 17 19>
status: After player 0 -> <1 3 1 7 1 1 1 5 17 19>
status: After player 1 -> <1 3 0 7 1 1 1 5 17 19>
status: After player 0 -> <1 3 0 7 1 1 1 0 17 19>
status: After player 1 -> <1 3 0 7 1 1 1 0 13 19>
status: After player 0 -> <1 3 0 7 1 1 1 0 13 1>
status: After player 1 -> <1 3 0 3 1 1 1 0 13 1>
status: After player 0 -> <1 3 0 3 1 1 1 0 1 1>
status: After player 1 -> <1 1 0 3 1 1 1 0 1 1>
status: After player 0 -> <1 1 0 1 1 1 1 0 1 1>
status: After player 1 -> <1 1 0 1 1 0 1 0 1 1>
status: After player 0 -> <1 1 0 1 1 0 0 0 1 1>
status: After player 1 -> <1 1 0 1 1 0 0 0 0 1>
status: After player 0 -> <1 1 0 1 1 0 0 0 0 0>
status: After player 1 -> <1 0 0 1 1 0 0 0 0 0>
status: After player 0 -> <0 0 0 1 1 0 0 0 0 0>
status: After player 1 -> <0 0 0 1 0 0 0 0 0 0>
status: After 